In [1]:
import seaborn as sns
sns.set_style('white')
flatui = ['#3498db', '#a6901b', '#e8a6ca','#a2b4c5', '#2ecc71', 
          '#d40b51', '#543cee']
sns.set_palette(flatui)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
sys.path.append('/home/jevjev/Dropbox/Projects/Intrinsic-dimensionality/')
from src.utils.train import get_exponential_range
from src.models import get_resnet
from torchvision.models import resnet18
import torch

In [2]:
resnet18_imagenet = resnet18(True)

In [3]:
resnet18_mean_var = get_resnet("resnet18", 10, pretrained=False)

In [4]:
mean_var_names = []
for name, param in resnet18_mean_var.named_parameters():
    mean_var_names.append((name, param.size()))
print(len(mean_var_names))

62


In [5]:
imagenet_names = []
for name, param in resnet18_imagenet.named_parameters():
    imagenet_names.append((name, param.size()))
print(len(imagenet_names))

62


In [6]:
mean_var_dict = resnet18_mean_var.state_dict()
mean_var_keys = list(resnet18_mean_var.state_dict().keys())

In [7]:
with torch.no_grad():
    for i, (key, param) in enumerate(resnet18_imagenet.state_dict().items()):
        if "num_batches_tracked" not in key:
            mean, std = torch.mean(param), torch.std(param)
            mean_var_key = mean_var_keys[i]
            param_mean_var = mean_var_dict[mean_var_key]
            mean_var_dict[mean_var_key] = torch.nn.init.normal_(torch.empty_like(param_mean_var), 
                                                                mean=mean, std=std)
resnet18_mean_var.load_state_dict(mean_var_dict)

<All keys matched successfully>